In [22]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
from collections import defaultdict
from keras_facenet import FaceNet


# Inisialisasi model YOLOv8
model = FaceNet()  # Ganti dengan path yang sesuai untuk model Anda
model.conf = 0.20  # Threshold confidence
model.maxdet = 10  # Maksimal deteksi

# Buka kamera
cap = cv2.VideoCapture(0)  # 0 untuk kamera default

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the camera
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Check if results contain boxes
        if results[0].boxes is not None:
            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist() if results[0].boxes.id is not None else []

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for the latest frames
                    track.pop(0)

                # Draw the tracking lines
                # points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                # cv2.polylines(
                #     annotated_frame,
                #     [points],
                #     isClosed=False,
                #     color=(230, 230, 230),
                #     thickness=2,  # Adjust thickness as needed
                # )
        else:
            # No detections found
            annotated_frame = frame  # Show the original frame

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the camera stream is not available
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [5]:
import cv2
import numpy as np
from keras.models import load_model
from keras_facenet import FaceNet
from collections import defaultdict

# Memuat model FaceNet yang telah dilatih
model_path = './model.h5'  # Ganti dengan path ke model Anda
model = load_model(model_path)

# Buka kamera
cap = cv2.VideoCapture(0)  # 0 untuk kamera default

# Store embeddings dan nama untuk pengenalan
known_embeddings = {}  # Simpan embeddings yang diketahui
track_history = defaultdict(lambda: [])

# Fungsi untuk mendeteksi dan mengenali wajah
def recognize_face(embeddings):
    # Di sini Anda perlu membandingkan embeddings dengan known_embeddings
    # Contoh sederhana untuk pengenalan, bisa diubah sesuai kebutuhan
    for name, known_embedding in known_embeddings.items():
        distance = np.linalg.norm(embeddings - known_embedding)  # Menghitung jarak
        if distance < 0.7:  # Threshold jarak untuk pengenalan
            return name  # Kembalikan nama jika ditemukan
    return "Unknown"  # Kembalikan "Unknown" jika tidak ada yang dikenali

# Loop melalui frame video
while cap.isOpened():
    # Baca frame dari kamera
    success, frame = cap.read()

    if success:
        # Ubah ukuran frame untuk memudahkan deteksi wajah
        small_frame = cv2.resize(frame, (160, 160))
        # Normalisasi gambar
        small_frame = small_frame / 255.0
        # Tambahkan batch dimension
        small_frame = np.expand_dims(small_frame, axis=0)

        # Menghitung embedding wajah menggunakan model FaceNet
        embeddings = model.predict(small_frame)

        # Kenali wajah berdasarkan embeddings
        name = recognize_face(embeddings[0])  # Menggunakan embeddings pertama

        # Gambar kotak dan label di frame asli
        h, w, _ = frame.shape
        cv2.rectangle(frame, (50, 50), (w - 50, h - 50), (0, 255, 0), 2)  # Gambar kotak
        cv2.putText(frame, name, (60, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  # Gambar label

        # Tampilkan frame
        cv2.imshow("FaceNet Detection", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the camera stream is not available
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


OSError: Unable to open file (file signature not found)